Hämta data från WIkidata
* [video](https://youtu.be/ZcvV4det8tA)
* denna [Notebook]()

In [2]:
# pip install sparqlwrapper
# https://rdflib.github.io/sparqlwrapper/

import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

query = """#title: partier i Sverige
SELECT distinct ?party ?partyLabel ?wikipedia ?www WHERE {

  VALUES ?member {
    wd:Q10655178 
  }
  ?person wdt:P39 ?member.
  ?person wdt:P102 ?party.
  ?party wdt:P17 wd:Q34.
  ?party wdt:P856 ?www
  minus {?party wdt:P576 ?xxx}
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv". }
  
   OPTIONAL {
      ?wikipedia schema:about ?party .
      ?wikipedia schema:inLanguage "sv" .
      ?wikipedia schema:isPartOf <https://sv.wikipedia.org/> .
    }
} GROUP BY  ?party ?partyLabel ?wikipedia ?www
order by ?partyLabel"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()


results = get_results(endpoint_url, query)

for result in results["results"]["bindings"]:
    print(result)


{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q50383811'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'Alternativ för Sverige'}, 'wikipedia': {'type': 'uri', 'value': 'https://sv.wikipedia.org/wiki/Alternativ_f%C3%B6r_Sverige'}, 'www': {'type': 'uri', 'value': 'https://alternativforsverige.se/'}}
{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q110832'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'Centerpartiet'}, 'wikipedia': {'type': 'uri', 'value': 'https://sv.wikipedia.org/wiki/Centerpartiet'}, 'www': {'type': 'uri', 'value': 'http://www.centerpartiet.se/'}}
{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q111449676'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal', 'value': 'Klimatalliansen'}, 'www': {'type': 'uri', 'value': 'https://klimatalliansen.nu/'}}
{'party': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q213654'}, 'partyLabel': {'xml:lang': 'sv', 'type': 'literal',